In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline 
import seaborn as sns

In [ ]:
print(os.listdir('../input'))



In [ ]:
soccer_df = pd.read_csv('../input/footballsoccer-players-market-value/mostvaluableplayers.csv', delimiter=',',index_col='name')
soccer_df.dataframeName = 'mostvaluableplayers.csv'



Lets take a look at the data set.

In [ ]:
soccer_df.head()

Doesnt look like we will need the '#' column and price column name seems a bit long.

In [ ]:
soccer_df.dtypes

In [ ]:
soccer_df.shape

In [ ]:
#Check for missing values
soccer_df.isnull().sum()

No missing values which means can start looking into the data.

In [ ]:
#Remove unnecessary column
soccer_df.drop(['#'],inplace=True,axis=1)

#Rename price column
soccer_df.rename(columns={'price_Mill._€': 'price'},inplace=True)

soccer_df.head()

In [ ]:
#Summary of Price
soccer_df['price'].describe(percentiles=[.25,.5,.75])

Mean is much close to the min which may indicate a very skewed price distribution.Lets take a look.

In [ ]:
#Distribution Plot of Price
sns.set(rc={'figure.figsize':(10,8)})
price_distn_plot = sns.distplot(soccer_df['price'], kde=False,bins = 20)
plt.title("Distribution Plot for Price")
price_distn_plot

As we saw in the summary, most players are getting paid under 50 euros(mill) and very few are getting paid over 100. Makes sense since only superstar players get top contracts. Lets look at which positions get paid the most on average.

In [ ]:
#Price by position
soccer_df.groupby(['position'])['price'].count().sort_values(ascending=False)

It may be hard to get an idea of which position gets paid more from these columns since the sample size is very small. In soccer there are 4 main positions (forward,midfield,defense,and goalkeeper). Im going to create a new column with just there general position. 

In [ ]:
#Create new column with general position for each player
soccer_df['general_pos'] = soccer_df.position

soccer_df['general_pos'].replace(to_replace=['Centre-Forward','Left Winger','Right Winger','Second Striker']\
                                 ,value='Forward',inplace=True)

soccer_df['general_pos'].replace(to_replace=['Central Midfield','Left Midfield','Right Midfield','Attacking Midfield','Defensive Midfield']\
                                 ,value='Midfield',inplace=True)

soccer_df['general_pos'].replace(to_replace=['Centre-Back','Left-Back','Right-Back','Second Striker']\
                                 ,value='Defense',inplace=True)

soccer_df.tail()

In [ ]:
#Price by General Position

soccer_df.groupby(['general_pos'])['price'].count().sort_values(ascending=False)

Each position has a much bigger sample size now with exception of goalkeeper since there is only one per team on the field. Lets compare the prices for each position.

In [ ]:
#Avg by General Position
soccer_df.groupby(['general_pos'])['price'].mean().sort_values(ascending=False)

Forward have a higher average than both midfield and defense. Goalkeeper is the highest but may not be very accurate since the sample size is only 20 one high paid goalkeeper may influence the average. Lets look at the price summary by position.

In [ ]:
#Forward Price
forward_df = soccer_df.loc[soccer_df['general_pos'] == 'Forward']
forward_price = forward_df[['price']]

#Midfield Price
mid_df = soccer_df.loc[soccer_df['general_pos'] == 'Midfield']
mid_price = mid_df[['price']]

#Defense Price
defense_df = soccer_df.loc[soccer_df['general_pos'] == 'Defense']
defense_price = defense_df[['price']]

#Keeper Price
keeper_df = soccer_df.loc[soccer_df['general_pos'] == 'Goalkeeper']
keeper_price = keeper_df[['price']]

#All price in a List
price_by_pos = [forward_price,mid_price,defense_price,keeper_price]

In [ ]:
#Empty list to append
p_b_p=[]

#List with values for column names
position = ['Forward','Midfield','Defense','Goalkeeper']

#for loop to get summary statistics for each position
for x in price_by_pos:
    i=x.describe(percentiles=[.25,.50,.75])
    i.reset_index(inplace=True)
    i.rename(columns={'index':'Stats'},inplace=True)
    p_b_p.append(i)
    

In [ ]:
#rename columns
p_b_p[0].rename(columns={'price':position[0]},inplace=True)
p_b_p[1].rename(columns={'price':position[1]},inplace=True)
p_b_p[2].rename(columns={'price':position[2]},inplace=True)
p_b_p[3].rename(columns={'price':position[3]},inplace=True)

#View Table
position_summary = p_b_p
position_summary =[df.set_index('Stats') for df in position_summary]
position_summary = position_summary[0].join(position_summary[1:])
position_summary

From the table forwards have a much higher std as well as mean ,again showing that forwards may be getting paid slightly more than other positions. Lets look at the density & distribution of each.

In [ ]:
#Violin plot to show distribution of prices for each position
sns.set(rc={'figure.figsize':(15,10)})
v_plot = sns.violinplot(x='general_pos',y='price',data=soccer_df,scale='area')
plt.title('Density & Distribution of price for each position')
v_plot

All distributions seem to be unimodal and peak around the same price. The difference comes in the tails of each. The forwards position is much more skewed indicating many outliers and higher paid players unlike defenders who dont seem to have any heavily payed players. The median looks lowest for the midfield position also with a small upper quartile when compared to the other positions. The third quartile of the goalkeeper box and whisker plot is longer than other poistions so there may be one or two players that have a big influence on the plot. Lers look at the highest paid players and there position.

In [ ]:
#Players getting paid atleast €100 
over_100 = soccer_df.loc[(soccer_df['price'] >= 100)]
over_100.groupby('general_pos').size()

In [ ]:
over_100

Jan Oblak is the reason why the Avg is higher for goalkeepers. 13 of the 18 players that get paid over 100 euro's are forwards. Seems reasonable since they usually score the most goals. Also its interesting to see that no player other than messi is over 30 years old in this list even thought there was almost no correlation between age & price(-.003).